In [25]:
from IPython.display import HTML, display
display(HTML('''
 <link rel="stylesheet" href="custom.css" type="text/css" />
'''))

In [26]:
!qr --factory=pymaging http://bit.ly/scipyla_scada2iot > img/qr.png

<center>
[![View this Presentation Online](img/qr.png?refresh)](http://bit.ly/scipyla_scada2iot)
</center>
View this presentation on Notebook Viewer

# From SCADA to IoT with Python

## Nahuel Defossé

<h3>
<a href="https://twitter.com/D3f0"><i class="fa fa-twitter" aria-hidden="true">&nbsp;</i></a>
<a href="https://github.com/D3f0"><i class="fa fa-github" aria-hidden="true">&nbsp;</i></a>
</h3>

# About me <i class="fa fa-info-circle" aria-hidden="true">&nbsp;</i>
 * Memmber of Research Group focused on Micronctollers and Netowrks.
 * Chief of Practical Assgnments at Universidad Nacional de la Patagonia
 * Python Advocate
     - Python courses in my local University
     - Regularlly give talks at [Patagonian Python Meetup](http://www.meetup.com/es/Patagonia-Python-Meetup/)
     - Chair at SciPy Latinamerica [2013](http://www.scipyla.org/conf/2013/) and [2014](http://www.scipyla.org/conf/2014/)

# Disclaimer
 * Not a good English speaker but definetly worse at Protuguese.
 * Some material cannot be distributed openly, approach me personaly to get more details.

# Owed credit <i class="fa fa-users" aria-hidden="true">&nbsp;</i>

 * Ricardo A **Lopez** (UNPSJB, Argentina)
 * Fernando A **Tinetti**(UNLP, Argentina)
 * Lautaro **Pecile** (UNPSJB, Argentina)
 * Diego **van Haaster** (UNPSJB, Argentina)
 * Sebastián **Wahler** (UNPSJB, Argentina)
 * Fernando **Páez** (UNPSJB, Argentina)
 * Manuel **Urriza** (UNS, Argentina)

# Talk organization

* Python for Hardware Interaction
* Python for Process Monitoring
* Python as tooling 
* Python in the microcontoller

# Python for Hardware Interaction

<img style="width: 70%" src="img/real_world_instumentation_w_python.jpg">
<p>Photo by <a href="https://www.flickr.com/photos/brewbooks/">brewbooks</a></p>

<!-- # SCADA <i class="fa fa-tachometer" aria-hidden="true">&nbsp;</i> -->
# Python for Process Monitoring (with Web UI)


# Requirements

* Monitor states and events
* Using insdustrial protocols
* Store in databases and do some calculations
* Show ovearall and in detail status (HMI)
* Generate reports

<img src="img/estructura.png">

<img src="img/estructura2.png">

### IED

<div>
<img style="float: left;" src="img/alpha2.png"/>
<img style="float: right;" src="img/ied.png"/>
</div>

# COMaster

<img src="img/comaster.png">

# Our specific requirements for electric systems


* Time sychronization
* State and event retrieval though POLL
* Commnds
* Designed for low cost 8 bit micro-controllers

# Wire protocol

<pre style="font-size: .7em;">
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┐┌─────┐┌─────┬─────┐
│ SOF │ SEQ │ QTY │ SRC │ DST │ CMD │ CMD ││ ... ││ BCL │ BCH │
│     │     │     │     │     │     │     ││     ││     │     │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┘└─────┘└─────┴─────┘
┌─────┬───────────┬───────────┬─────┬───────────┬───────────┐  
│ QSV │ SV0       │ SV1       │ QDI │ DI0       │ DI1       │  
│     │           │           │     │           │           │  
└─────┴───────────┴───────────┴─────┴───────────┴───────────┘  
┌─────┬───────────┬───────────┬─────┬─────────────────────────┐
│ QAI │ AI0       │ AI1       │ QEV │ EV..                    .
│     │           │           │     │                         │
└─────┴───────────┴───────────┴─────┴─────────────────────────┘
</pre>

# Parsing Mara Frame 

 * `pip install construct`
 * Forget about `struct`, Constrct gives a DSL for binary parsing.

In [41]:
from construct import *

PascalString = Struct("PascalString",
    UBInt8("length"),
    Bytes("data", lambda ctx: ctx.length),
)
PascalString.parse(b"\x05helloXXX")

Container({'length': 5, 'data': b'hello'})

# [Mara Definition]()

In [44]:
MaraFrame = BaseMaraStruct(
    'Mara',
    ULInt8('sof'),
    ULInt8('length'),
    ULInt8('dest'),
    ULInt8('source'),
    ULInt8('sequence'),
    ULInt8('command'),
    # Probe(),

    If(lambda ctx: ctx.command == 0x10,
       # Probe(),
       Optional(Payload_10),
    ),

    If(lambda ctx: ctx.command == 0x12,
       PEHAdapter(Payload_PEH),
    ),
    ULInt16('bcc')
)

NameError: name 'BaseMaraStruct' is not defined

# Data Storage

* Initially written with SQLAlchemy and Flask
* Migrated to Django due to ORM, Admin, REST 3rd party apps and management commands.

<img src="img/mara.svg">

# [Saving data on DB](https://github.com/D3f0/txscada/blob/master/src/pysmve/nguru/apps/mara/models.py#L200)

In [48]:
# DI
for value, di in zip(iterbits(payload.dis, length=16), self.dis):
    # Update value takes care of maskinv
    di.update_value(value, q=0, last_update=timestamp)
    di_count += 1
# AI
for value, ai in zip(payload.ais, self.ais):
    value = value & 0x0FFF
    q = (value & 0xC000) >> 14
    ai.update_value(value, last_update=timestamp, q=q)
    ai_count += 1
# ...

NameError: name 'iterbits' is not defined

# Human Machine Interface (HMI)



![](img/smve1.png)

# SVG

* Each *mutable* element has an ID **SVGElement** and belongs to a **Screen** (SVG file).
* A **Screen** can have a parent and might have links to other screens (tree structure).
* Each **SVGElement** SVG attribute has a formula associated. 
* When a new value is read from the field, the formula is calculated and the property refreshed on screen.
* This formulas are a Pythonish excel syntax, and are loaded into the DB with Pandas' [`read_excel`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)

<img src="img/hmi.svg">

# Formula

```python
SI(di.E2152B04.value,si(int(eg.E2189S04.text),eg.E2189S04.stroke,12),si(int(eg.E2152B04.text),6,10))
```


## Moving to a less structured scenario
* Publisher/Subscriber instead of Polling
* Bigger and more deiverse enviroments
* **Arduino API** compatible hardware 


# PlatformIO

<center>
<img src="img/platformio-logo.png">
</center>

# Supported PlatformIO boards

<img src="img/plataformas.png">

<div>
<img src="img/arduino-ubuntu.jpg" style="width: 40%; float: left; margin-top: 4%;">
<img src="img/ide.png" style="width: 40%; float: right">
</div>

In [52]:
!open img

**CoAP** inherits many concepts from HTTP <br/>
**MQTT** Message Queues, QoS.

# Conclusions

* Django ORM might not be the best way to integrate things
* Not loosing any package has to be considered in the wire protocol
* Design the system for fault tolerance
* Python dynamic natu

In [54]:
!git add img && git add -u && git commit -m "Updates $(date)" && git push origin master

[master 8d1619f] Updates Fri May 20 01:21:48 ART 2016
 1 file changed, 57 insertions(+), 9 deletions(-)
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.07 KiB | 0 bytes/s, done.
Total 6 (delta 2), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   cbd5bb7..8d1619f  master -> master


In [8]:
!open custom.css

In [34]:
!open .